# LSST Catalog Access Tutorial

This notebook provides a simple demonstration on how to access the catalog data for LSST. It is primarily provided for system testing. For science-centered tutorial notebooks please see the Rubin Community Science Team's collection at https://github.com/rubin-dp0/tutorial-notebooks

We use the IVOA TAP (Table Access Procotol) standard to access the catalog data.  This standardizes usage so that 3rd party clients and astronomers can easily move their queries around between datasets, and discover what data is available for them to query.

Here's a link to the standard: http://www.ivoa.net/documents/TAP/

The TAP service uses a query language similar to SQL called ADQL.  For the specifics about the syntax and keywords, look at: http://www.ivoa.net/documents/latest/ADQL.html

### TOPCAT usage

Some astronomers prefer to use tools like TOPCAT to access the catalogs.  In order to connect TOPCAT to the TAP service, you will need to provide a service endpoint for TOPCAT to use.  Hou can find the service endpoint for the environment you are using by executing the cell in [Section 1](#1.-Create-the-client).  If you can contact the endpoint printed by that cell from your network, you can also use this as the "Selected TAP Service" in TOPCAT.

See [this](https://nb.lsst.io/environment/tokens.html) page for more in formation on using the TAP service with TOPCAT.  Note that those instructions assume a TAP service running at NCSA, so be sure to substitute the endopint for your specific environment as provided below for the NCSA specific one in the documentation.

### Portal usage

The TAP service is used by the portal and users can use the Portal UI to help them construct queries.

### Notebook Usage

In the following steps, we will create a client to query the catalog, show how you can discover what tables and columns exist in the catalog, as well as how to run queries and process their results using python.

For this example, we will be using pyvo: https://pyvo.readthedocs.io/en/latest/index.html

## 1. Create the client

First, you must retrieve the TAP service object to use for querying the data.  This will be the object you call to run queries on.  Run the cell below to create your client.

In [ ]:
import pandas
from IPython.display import Markdown as md

from lsst.rsp import get_tap_service, retrieve_query

pandas.set_option('display.max_rows', 1000)
service = get_tap_service("tap")

md(f'The service endpoint for TAP in this environment is:\n\n &#10145;&nbsp;&nbsp;   {service.baseurl}')

# 2. Schema discovery

Now that you've connected to the TAP service, you need to know the names of schemas, tables, and columns (and their datatypes) to be able to write queries to access the data.

Luckily, TAP provides for a standard way of discovering what schemas, tables, and columns are available.  This information is stored in a database named TAP_SCHEMA.

### 2.1 What schemas can I query?

In order to discover what database schemas are being served and can be queried against, you can run the following cell.  You should see a numpy compatible table printed out in the notebook that has two important columns, description and schema_name.  

In [ ]:
results = service.search("SELECT schema_name, description FROM TAP_SCHEMA.schemas")
results.to_table().show_in_notebook()

**TAP_SCHEMA.schemas** behaves like a normal SQL table, with mostly the same query language. Just like standard SQL, you can also retrieve all the columns by using the following example.

**TIP:** Calling to_table() on a result will give you an AstroPy table (http://docs.astropy.org/en/stable/_modules/astropy/table/table.html).

From an astropy table, you can also convert to a pandas dataframe by using to_pandas().

In [ ]:
results = service.search("SELECT * FROM TAP_SCHEMA.schemas")
results.to_table().show_in_notebook()

### 2.2 What tables can I query?

Like the TAP_SCHEMA.schemas table, there is also a **TAP_SCHEMA.tables** table, which contains the names of the tables, and which schema each table belongs to.

In [ ]:
results = service.search("SELECT * FROM TAP_SCHEMA.tables")
results.to_table().show_in_notebook()

Sometimes, you may want to act programmatically on the results.  There are also functions you can call on the results object to get the columns / fields, and iterate through the rows, as demonstrated below.

In [ ]:
print("Column names for TAP_SCHEMA.tables are:")
for field in results.fieldnames:
    print(field)

We can now use the column names to limit the columns returned by the query, as seen below.

In [ ]:
results = service.search("SELECT schema_name, table_name FROM TAP_SCHEMA.tables")
results.to_table().show_in_notebook()

### 2.3 What columns are in a table, and what datatype is each column?

Now that we've learned what tables exist, we're close to querying for real catalog data.  But generally, we want to limit the number of columns to return to exactly what we need to get the job done.  This generally makes things faster and better for everyone.  While you can get all the columns that exist (with a "SELECT * from TAP_SCHEMA.columns"), this will be over all the tables in all the schemas, so normally we want to limit the query to a particular table.

Let's discover what columns exist in the dp02_dc2_catalogs.DiaObject table using the following code.

**TIP**: Be careful with your quoting, since some quotes are processed by python, and some quotes are passed down as a part of the query.  My suggestion is to use double quotes for python, and single quotes for SQL.

In [ ]:
results = service.search("SELECT * from TAP_SCHEMA.columns WHERE table_name = 'dp02_dc2_catalogs.DiaObject'")

print("Column names for TAP_SCHEMA.columns are:")
print(results.fieldnames)

print("Columns that exist in dp02_dc2_catalogs.DiaObject are:")
results.to_table().show_in_notebook()

The most useful columns in TAP_SCHEMA.columns are the column_name, unit, and description.  Let's just query those.

In [ ]:
results = service.search("SELECT column_name, description, unit FROM TAP_SCHEMA.columns WHERE table_name = 'dp02_dc2_catalogs.DiaObject'")
results.to_table().show_in_notebook()

# 3. Example queries

Here are some example queries against the DC02 dataset.  As more queries are supported, they will be added here.

**Note**: Not all of these are against the same table, or against the same columns.  These queries are simply for reference.

### 3.1 Cone searches

How to do a cone search (with a center RA/DEC, and a radius):

The format is CONTAINS(POINT('ICRS', dec column or number, ra column or number), CIRCLE('ICRS', center RA, center DEC, radius)) = 1.  All units are in degrees.

**TIP**: You can limit the number of rows returned by using maxrec=x as a parameter.  This means it will only return x rows.

In [ ]:
results = service.search("SELECT ra, decl FROM dp02_dc2_catalogs.DiaObject WHERE CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 60.0, -30.0, 0.05)) = 1", maxrec=10)
results.to_table().show_in_notebook()

### 3.2 Polygon searches

Sometimes cone search isn't enough.  Sometimes you want to submit a set of arbitrary vertices and draw your own polygon to limit to.  You can use the POLYGON function.  This behaves like CIRCLE, but allows for any number of RA/DEC coordinate pairs to be vertices.  The polygon is automatically closed between the last vertex and the first vertex.

The format is CONTAINS(POINT('ICRS', dec column or number, ra column or number), POLYGON('ICRS', RA 1, DEC 1, RA 2, DEC 2, ...)) = 1. All units are in degrees.

Following is an example of using POLYGON to make a square like shape.

In [ ]:
results = service.search("SELECT ra, decl FROM dp02_dc2_catalogs.DiaObject WHERE CONTAINS(POINT('ICRS', ra, decl), POLYGON('ICRS', 60.0, -30.0, 60.05, -30, 60.05, -30.05, 60, -30.05)) = 1", maxrec=10)
results.to_table().show_in_notebook()

# 4 Asynchronous queries

So far, all the queries you have run are called a synchronous queries.  This means that it will continue executing in the notebook until it is finished.  You can see when a Jupyter cell is running because it will have an asterisk to the left of the running cell.  When the query is finished executing and results are returned, the asterisk will become a number.  This is great for short queries that may take seconds or a minute.

For longer queries, or running multiple queries at the same time, you may want to consider running an asynchronous query.  Asynchronous queries can be started and allow you to execute more python while the query runs on the database.  Results can be retrieved later on.  This is especially important for queries that are long or may return a lot of results.

### 4.1 Asynchronous query example
Let's run one of these queries asynchronously:

In [ ]:
# First, submit the job.  This creates it but doesn't run the query yet.
job = service.submit_job("SELECT schema_name, description FROM TAP_SCHEMA.schemas")

# Here's the URL representing your query.  This is what you need to retrieve
# the data again.
print('Job URL is', job.url)

# Here you can see the job's phase, which is PENDING.
print('Job phase is', job.phase)

# Now, run the query.
job.run()

# Here, you can tell python to wait for the job to finish if you don't want to
# run anything else while you are waiting.  Then we print out the final state.
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

# Here's a helpful function that will raise an exception if
# your query had an unfortunate incident.
job.raise_if_error()

# Once it completes successfully, you can work with the results
# the same exact way as if it were synchronous. 
results = job.fetch_result()
results.to_table().show_in_notebook()

## 4.2 Retrieving results from a previous query

If you have already run a query, the reuslts may still be available.  You can easily retrieve the results from a previous query again if you know the URL of the query.

You can use the query URL to do a few things:
1. Use the File -> Open Query ID and put in the URL to start a notebook to load your results.
2. Use TOPCAT TAP Query tool to load the query results into TOPCAT.  Use the "Resume Job" tab in the TAP Service tool.
3. Use the retrieve_query function from jupyterhubutils.notebook to retrieve the results.

Here we will show you the last one to load the same query results as the query we just ran above:

In [ ]:
retrieved_job = retrieve_query(job.url)
results = retrieved_job.fetch_result()
results.to_table().show_in_notebook()

### 4.3 Deleting results

Once your asynchronous job is finished, the results are stored on the server until you are ready to retrieve them, as shown in the above section.  When you are done with your results, you can delete them yourself, or the server will delete results automatically after a period of time.  After a job is deleted, you will have to re-run the original query on the server to get the results again.

In [ ]:
job.delete()

### 4.4 Long running queries running as asynchronous

Even if you don't have something else to do in the background, sometimes you will want to run queries that take a long time asynchronously.  This is because it is safer from network outages and timeouts to run it that way.  Luckily, there's a helper function to run a query asynchronously and wait for it to finish, just like a synchronous query.  It basically does what we've done above.  run_async will also automatically delete the results after they are received.

Let's try that:

In [ ]:
results = service.run_async("SELECT schema_name, description FROM TAP_SCHEMA.schemas")
results.to_table().show_in_notebook()

# 5. Future work / Known issues

The work for this service is ongoing, and there will be known limitations and bugs.  If you want to contact someone for help, please go to the #dm-rsp-team channel on the LSST Slack.

### 5.1 Known limitations

This TAP layer is built on top of QServ, which has its own query limitations that may come up.  Here is a document outlining those restrictions:

Some ADQL functions are not yet supported:

- AREA
- BOX
- COORDSYS
- COORD1
- COORD2
- INTERSECTS

The only coordinate system currently supported is 'ICRS'.

https://github.com/lsst/qserv/blob/master/UserManual.md

### 5.2 Performance

If your query takes more than a minute or so, it's likely you have run a query that is enacting a full table scan.  This can take a long time (~ hour) to return.  Please be patient and don't re-run your query as this may put the database under additional load.